# Technical Factor - Relative Price Strength (RPS)

In [13]:
import dai
import pandas as pd

In [14]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [15]:
sql = f"""
WITH
data_factor AS (
    WITH
    data_stock AS (
        SELECT
            date,
            instrument,
            (close - m_lag(close, 6)) / m_lag(close, 6) AS security_return
        FROM cn_stock_bar1d
    ),
    data_index AS (
        SELECT
            date,
            (close - m_lag(close, 6)) / m_lag(close, 6) AS index_return
        FROM cn_stock_index_bar1d
        WHERE instrument = '000300.SH'
    ),
    data_merge AS (
        SELECT
            date,
            instrument,
            security_return,
            index_return,
            (security_return / (index_return + 0.0001)) * 100 AS RPS
        FROM data_stock
        JOIN data_index USING (date)
    )
    SELECT
        date,
        instrument,
        RPS,
        security_return,
        index_return
    FROM data_merge
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(RPS > 100 AND m_lag(RPS, 1) <= 100, 1, 0) AS TRBY1,
        IF(RPS < 100 AND m_lag(RPS, 1) >= 100, 1, 0) AS TRSL1
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(RPS > 100 AND (RPS - m_lag(RPS, 1)) > 0, 1, 0) AS MTBY1,
        IF(RPS < 100 AND (RPS - m_lag(RPS, 1)) < 0, 1, 0) AS MTSL1
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(RPS < 80 AND (RPS - m_lag(RPS, 1)) > 0, 1, 0) AS RVBY1,
        IF(RPS > 120 AND (RPS - m_lag(RPS, 1)) < 0, 1, 0) AS RVSL1
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(RPS, 1) <= 120 AND RPS > 120, 1, 0) AS BKBY1,
        IF(m_lag(RPS, 1) >= 80 AND RPS < 80, 1, 0) AS BKSL1
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        RPS,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
AND instrument IN {str(tuple(instrument_list))}
ORDER BY date, instrument
"""

In [16]:
df = dai.query(sql, filters={"date":[sd,ed]}).df()
df

,date,instrument,RPS,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-10,600519.SH,152.524311,0,0,0,0,0,0,0,0
1,2025-01-13,600519.SH,151.878955,0,0,0,0,0,1,0,0
2,2025-01-14,600519.SH,163.751609,0,0,1,0,0,0,0,0
3,2025-01-15,600519.SH,27288.235700,0,0,1,0,0,0,0,0
4,2025-01-16,600519.SH,88.264830,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
232,2025-12-25,600519.SH,26.082286,0,0,0,0,1,0,0,0
233,2025-12-26,600519.SH,21.869108,0,0,0,1,0,0,0,0
234,2025-12-29,600519.SH,-36.173685,0,0,0,1,0,0,0,0
235,2025-12-30,600519.SH,-151.330207,0,0,0,1,0,0,0,0
